In [3]:
import gurobipy as gp
import pandas as pd

# read excel sheets
data = pd.ExcelFile('SuperChipData.xlsx')
productionCapacityDf = pd.read_excel(data, 'Production Capacity')
salesRegionDemandDf = pd.read_excel(data, 'Sales Region Demand')
shippingCostsDf = pd.read_excel(data, 'Shipping Costs')
ProductionCostsDf = pd.read_excel(data, 'Production Costs')

# facility S
facilitySet = {"Alexandria", "Richmond", "Norfolk", "Roanoke", "Charlottesville"}
# sales region j
salesRegion = 23
# computer chip z
computerChip = 30
# production capacity C
capacity = [348, 312, 294, 216, 204]
# Djz = z chip yearly demand in sales region j (thousands unit)
Djz = salesRegionDemandDf.to_numpy()
# Ci = yearly chip production capacity in facility i (thousands unit)
Ci = productionCapacityDf.to_numpy()
# Piz = production cost per z chip in facility i ($)
Piz = ProductionCostsDf.to_numpy()
# Sijz = shipping cost per z chip from facility i to sales region j ($)
Sijz = shippingCostsDf.to_numpy()

m = gp.Model()
m.modelSense = gp.GRB.MINIMIZE

[['Alexandria' 1 59.79]
 ['Alexandria' 2 52.8]
 ['Alexandria' 3 58.2]
 ['Alexandria' 4 44.46]
 ['Alexandria' 5 52.71]
 ['Alexandria' 6 48.07]
 ['Alexandria' 7 58.85]
 ['Alexandria' 8 52.31]
 ['Alexandria' 9 44.45]
 ['Alexandria' 10 46.35]
 ['Alexandria' 11 46.3]
 ['Alexandria' 12 40.54]
 ['Alexandria' 13 50.4]
 ['Alexandria' 14 59.14]
 ['Alexandria' 15 41.24]
 ['Alexandria' 16 53.84]
 ['Alexandria' 17 51.06]
 ['Alexandria' 18 58.37]
 ['Alexandria' 19 45.91]
 ['Alexandria' 20 44.73]
 ['Alexandria' 21 51.57]
 ['Alexandria' 22 52.91]
 ['Alexandria' 23 56.54]
 ['Alexandria' 24 42.62]
 ['Alexandria' 25 52.19]
 ['Alexandria' 26 47.02]
 ['Alexandria' 27 48.53]
 ['Alexandria' 28 53.09]
 ['Alexandria' 29 57.49]
 ['Alexandria' 30 52.24]
 ['Richmond' 1 46.73]
 ['Richmond' 2 45.51]
 ['Richmond' 3 59.34]
 ['Richmond' 4 54.74]
 ['Richmond' 5 43.26]
 ['Richmond' 6 47.53]
 ['Richmond' 7 54.34]
 ['Richmond' 8 52.52]
 ['Richmond' 9 58.85]
 ['Richmond' 10 43.23]
 ['Richmond' 11 59.09]
 ['Richmond' 12 46.